# 📊 Bayesian Analysis of Curvality Residuals
This notebook compares your entropy-curvature oscillation fit against a null hypothesis using real CMB TE/EE residual data.

In [ ]:
# 📦 Install required packages
!pip install pymc3 arviz --quiet

In [ ]:
# 📁 Load your real residual data
import pandas as pd
import numpy as np

# Upload all three files manually in Colab session when prompted
from google.colab import files
uploaded = files.upload()

df = pd.read_csv('Residuals_with_Oscillation_Fit.csv')
ell = df['ell'].values
residuals = df['residuals'].values
osc_fit = df['osc_fit'].values

In [ ]:
# 📈 Visualize the data
import matplotlib.pyplot as plt
plt.plot(ell, residuals, label='Observed Residuals')
plt.plot(ell, osc_fit, label='Oscillation Fit', linestyle='--')
plt.xlabel('Multipole ℓ')
plt.ylabel('Residuals (µK²)')
plt.legend()
plt.title('Curvality Residuals vs Oscillation Fit')
plt.grid(True)
plt.show()

In [ ]:
# 🔍 Bayesian Comparison
import pymc3 as pm
import arviz as az

with pm.Model() as model:
    amp = pm.Normal("amplitude", mu=1, sigma=1)
    phase = pm.Uniform("phase", 0, 2*np.pi)
    freq = pm.Normal("frequency", mu=0.1, sigma=0.05)
    baseline = pm.Normal("baseline", mu=0, sigma=1e-13)

    expected = amp * np.sin(freq * ell + phase) + baseline
    sigma = np.std(residuals - osc_fit)
    observed = pm.Normal("obs", mu=expected, sigma=sigma, observed=residuals)

    trace = pm.sample(2000, tune=1000, target_accept=0.9, return_inferencedata=True)
    az.plot_posterior(trace)
    plt.show()

    az.summary(trace, var_names=["amplitude", "frequency", "phase", "baseline"])